https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/

In [2]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [3]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
ERROR: Could not find a version that satisfies the requirement torch_nightly
ERROR: No matching distribution found for torch_nightly


In [4]:
path = untar_data(URLs.IMDB_SAMPLE)
path

WindowsPath('C:/Users/user/.fastai/data/imdb_sample')

In [5]:
path.ls()  #ดูว่ามีไฟล์อะไรบ้าง

[WindowsPath('C:/Users/user/.fastai/data/imdb_sample/texts.csv')]

In [6]:
df = pd.read_csv(path/'texts.csv')  #โหลดไฟล์ csv ขึ้นมา ใส่ Pandas Dataframe
df.shape

(1000, 3)

#มีข้อมูลตัวอย่าง 1000 Record, 3 Column ดูตัวอย่างข้อมูล 5 Record แรก
df.head()

Preprocessing

In [ ]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text').split_from_df(col=2).label_from_df(cols=0))
 #เราจะใช้ Fastai TextList ในการ Preprocess ข้อมูล เช่น Tokenization, Numberization, etc.
 #จะได้ Trainint Set, Validation Set ออกมา เรามาดูข้อมูลตัวอย่าง Record แรกใน Validation Set

https://medium.com/ai-in-plain-english/fast-natural-language-processing-101-cf1afefdc449

In [ ]:
movie_reviews.valid.y[0], movie_reviews.valid.x[0]
